# Sesiunea 1 – Inițializare Chat (Foundry Local)

Acest notebook inițializează Foundry Local, descarcă aliasul modelului preferat și efectuează atât o completare standard, cât și una de tip streaming pentru chat.


# Scenariu
Această sesiune introduce minimul absolut necesar pentru a obține un model lingvistic local mic care răspunde prin Foundry Local. Vei:
- Instala SDK-ul / dependențele clientului.
- Inițializa managerul Foundry Local pentru un alias ales (implicit: `phi-3.5-mini`).
- Aplica un monkey-patch defensiv pentru a tolera câmpurile opționale din metadata modelului.
- Trimite o cerere standard de completare a conversației.
- Transmite un răspuns, token cu token.

Scopul este să validezi runtime-ul local și calea de rețea înainte de a trece la RAG, rutare sau agenți.


### Explicație: Instalarea Dependențelor
Instalează pachetele Python necesare pentru acest flux minim de chat:
- `foundry-local-sdk`: Gestionarea modelelor locale și a ciclului de viață al serviciilor.
- `openai`: Abstracție familiară pentru completările de chat.
- `rich`: Afișare estetică pentru o ieșire mai clară în notebook.

Reexecutarea este sigură (idempotentă). Săriți peste acest pas dacă mediul dvs. are deja aceste pachete.


In [1]:
# Install required libraries (idempotent)
!pip install -q foundry-local-sdk openai rich

### Explicație: Importuri de Bază
Adaugă module utilizate pe parcursul notebook-ului:
- `FoundryLocalManager` pentru a interacționa cu runtime-ul local al modelului.
- Clientul `OpenAI` pentru a putea reutiliza suprafața familiară a API-ului de completare chat.
- `rich.print` pentru afișare stilizată.

Nu se efectuează apeluri de rețea aici—acest pas doar pregătește spațiul de nume.


In [2]:
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI
from rich import print

### Explicație: Inițializarea Managerului & Patch-ul Metadata
Inițializează `FoundryLocalManager` pentru aliasul ales și aplică un patch defensiv pentru a gestiona elegant răspunsurile serviciului în care `promptTemplate` ar putea fi `null`.

Rezultate cheie:
- Confirmă starea serviciului și endpoint-ul.
- Listează modelele cache (verifică stocarea locală).
- Rezolvă ID-ul concret al modelului pentru alias (utilizat în apelurile ulterioare de chat).

Dacă întâmpinați probleme de validare în metadatele brute ale serviciului, acest model arată cum să le sanitizați fără a modifica SDK-ul.


In [3]:
# Catalog-safe manager initialization (handles null promptTemplate values)
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-3.5-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:57127/v1

Cached models:
[
    FoundryModelInfo(
        alias='gpt-oss-20b',
        id='gpt-oss-20b-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/gpt-oss-20b-cuda-gpu/versions/1',
        file_size_mb=9882,
        prompt_template={},
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-cuda-gpu/versions/1',
        file_size_mb=2181,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-cuda-gpu:4',
        version='4',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-cuda-gpu/versions/4',
        file_size_mb=3686,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-cuda-gpu/versions/3',
        file_size_mb=528,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-7b',
        id='qwen2.5-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-coder-7b',
        id='qwen2.5-coder-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-coder-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_star

Using model id: Phi-4-mini-instruct-cuda-gpu:4

### Explicație: Completare de Chat de Bază
Creează un client compatibil `OpenAI` care indică spre endpoint-ul local Foundry și efectuează o singură completare de chat fără streaming. Puncte de interes:
- Asigură-te că modelul răspunde fără erori.
- Validează latența / formatul de ieșire.
- Menține `max_tokens` modest pentru a conserva resursele.

Dacă acest lucru eșuează, verifică din nou dacă serviciul Foundry Local este activ și aliasul se rezolvă corect.


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':prompt}], max_tokens=120, temperature=0.5)
print(resp.choices[0].message.content)

1. Enhanced Data Privacy: Local inference allows data to be processed locally on a user's device rather than being 
sent to a central server. This reduces the risk of sensitive information being intercepted or exposed during 
transmission, thereby enhancing privacy.

2. Reduced Data Collection: By performing inference locally, the need for extensive data collection is minimized. 
This means that less personal information needs to be stored or transmitted, further protecting user privacy.

### Explicație: Finalizare Chat în Flux
Demonstrează transmiterea de tokenuri în flux pentru o latență percepută mai bună și o experiență interactivă îmbunătățită. Bucla afișează incremental modificările pe măsură ce acestea sosesc:
- Util pentru interfețele de chat unde contează afișarea timpurie a unui rezultat parțial.
- Permite măsurarea debitului de tokenuri în raport cu latența completării totale.

Poți adapta acest model pentru a acumula tokenuri, a actualiza un widget de progres sau a întrerupe generarea la jumătate.


In [5]:
# Streaming example
stream = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}], stream=True, max_tokens=60, temperature=0.4)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

with

a

central

server

.


---

**Declinarea responsabilității**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să rețineți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa maternă ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm răspunderea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
